In [48]:
import re
import spacy
import nltk
import numpy as np
import pickle
from nltk.corpus import stopwords
from nltk import ngrams
import pymorphy3
import warnings
from spacy.tokenizer import Tokenizer
from tqdm import tqdm


warnings.filterwarnings('ignore')
nlp = spacy.load("ru_core_news_sm")
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
with open("data/russian_stopwords.txt", 'r', encoding='utf-8') as file:
    stop_words = file.read().split()


with open("data/Kamenev_Maks-Volf_2_Naemnik_RuLit_Me.txt", 'r') as file:
    data = file.read()
    
data[:155]

'\n   Алекс Каменев\n   Макс Вольф 2.\n   Наемник.\n   Глава 1.\n   Станция технического обслуживания «Бринга-17», Лиманский Союз, приграничная территория Содруж'

In [20]:
url_pattern = r'http[s]?://\S+|www\.\S+'
regex = re.compile(url_pattern)
data = regex.sub('', data)

data = re.sub(r'\b\d[\w\-\.]*', '', data)
data = re.sub(r'\b\-|\-\b', '', data)
data[:155]

'\n   Алекс Каменев\n   Макс Вольф \n   Наемник.\n   Глава \n   Станция технического обслуживания «Бринга», Лиманский Союз, приграничная территория Содружества.\n'

In [22]:
data = " ".join(data.lower().strip().split())
data[:155]

'алекс каменев макс вольф наемник. глава станция технического обслуживания «бринга», лиманский союз, приграничная территория содружества. то есть как, забло'

### <center>First method

In [25]:
def custom_tokenizer(nlp):
    prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
    infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
    suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)

    infix_re = spacy.util.compile_infix_regex([pattern for pattern in nlp.Defaults.infixes if '-' not in pattern])

    return Tokenizer(
        nlp.vocab,
        rules=nlp.Defaults.tokenizer_exceptions,
        prefix_search=prefix_re.search,
        suffix_search=suffix_re.search,
        infix_finditer=infix_re.finditer,
        token_match=None,
    )


def lemmatize_word_list(text):
    word_list = text.split()
    return [
        nlp(word)[0].lemma_
        for word in tqdm(word_list, desc="Лемматизация слов")
        if not nlp(word)[0].is_stop and not nlp(word)[0].is_punct
    ]

nlp.tokenizer = custom_tokenizer(nlp)
result1 = lemmatize_word_list(data)

Лемматизация слов: 100%|███████████████████████████████████████████████████████| 108331/108331 [18:06<00:00, 99.72it/s]


In [27]:
np.array(result1)

array(['алекс', 'каменев', 'макс', ..., 'дорасских', 'процент', 'акция'],
      dtype='<U35')

### <center>Second method

In [30]:
def custom_tokenizer(text):
    return re.findall(r'\b[\w\-]+\b', text)


def lemmatize_word_list2(text):
    tokens = custom_tokenizer(text)
    lemmatized = [
        morph.parse(token)[0].normal_form
        for token in tqdm(tokens, desc="Лемматизация слов №2")
        if token not in stop_words_list and re.search(r'\w', token)
    ]
    return lemmatized


stop_words_list = list(set(stop_words + russian_stopwords))
morph = pymorphy3.MorphAnalyzer()
result2 = lemmatize_word_list2(data)

Лемматизация слов №2: 100%|█████████████████████████████████████████████████| 106024/106024 [00:08<00:00, 11868.46it/s]


In [32]:
np.array(result2)

array(['алекс', 'каменеть', 'макс', ..., 'маркиз', 'дорасский', 'акция'],
      dtype='<U27')

### <center>Bag of Words

In [35]:
def create_BoW(tokens):
    word_counts = {}
    for token in tokens:
        if token in word_counts:
            word_counts[token] += 1
        else:
            word_counts[token] = 1

    sorted_word_counts = dict(sorted(word_counts.items(), key=lambda x: x[1], reverse=True))
    
    return sorted_word_counts


word_counts1 = create_BoW(result1)
word_counts2 = create_BoW(result2)
print("BoW1:")
display(list(word_counts1.items())[:10])
print("\nBoW2:")
display(list(word_counts2.items())[:10])

BoW1:


[('человек', 369),
 ('время', 266),
 ('корабль', 254),
 ('сторона', 208),
 ('новый', 205),
 ('рука', 196),
 ('планета', 193),
 ('два', 191),
 ('сказать', 190),
 ('дело', 187)]


BoW2:


[('корабль', 255),
 ('человек', 242),
 ('планета', 215),
 ('сторона', 206),
 ('мой', 189),
 ('канваля', 189),
 ('рука', 180),
 ('новый', 170),
 ('баронство', 169),
 ('тисара', 166)]

#### В случае с SpaCy (первым вариантом), скорее всего, происходит большая агрегация слов, и их частотность более высока, потому что лемматизация SpaCy может приводить к нормализации множества различных форм в одну лемму.
#### В pymorphy3 (второй версии) лемматизация может быть немного менее агрессивной, оставляя больше уникальных вариантов для некоторых слов, что приводит к меньшему числу повторений.

### <center>2-Grams

In [39]:
bigrams1 = list(ngrams(result1, 2))
print(f"2-граммы1: {bigrams1[:10]}")
bigrams2 = list(ngrams(result2, 2))
print(f"\n2-граммы2: {bigrams2[:10]}")

2-граммы1: [('алекс', 'каменев'), ('каменев', 'макс'), ('макс', 'вольф'), ('вольф', 'наёмник'), ('наёмник', 'глава'), ('глава', 'станция'), ('станция', 'технический'), ('технический', 'обслуживание'), ('обслуживание', 'лиманский'), ('лиманский', 'союз')]

2-граммы2: [('алекс', 'каменеть'), ('каменеть', 'макс'), ('макс', 'вольф'), ('вольф', 'наёмник'), ('наёмник', 'глава'), ('глава', 'станция'), ('станция', 'технический'), ('технический', 'обслуживание'), ('обслуживание', 'бринг'), ('бринг', 'лиманский')]


### <center>4-Grams

In [42]:
fourgrams1 = list(ngrams(result1, 4))
print(f"4-граммы1: {fourgrams1[:10]}")
fourgrams2 = list(ngrams(result2, 4))
print(f"\n4-граммы2: {fourgrams2[:10]}")

4-граммы1: [('алекс', 'каменев', 'макс', 'вольф'), ('каменев', 'макс', 'вольф', 'наёмник'), ('макс', 'вольф', 'наёмник', 'глава'), ('вольф', 'наёмник', 'глава', 'станция'), ('наёмник', 'глава', 'станция', 'технический'), ('глава', 'станция', 'технический', 'обслуживание'), ('станция', 'технический', 'обслуживание', 'лиманский'), ('технический', 'обслуживание', 'лиманский', 'союз'), ('обслуживание', 'лиманский', 'союз', 'приграничный'), ('лиманский', 'союз', 'приграничный', 'территория')]

4-граммы2: [('алекс', 'каменеть', 'макс', 'вольф'), ('каменеть', 'макс', 'вольф', 'наёмник'), ('макс', 'вольф', 'наёмник', 'глава'), ('вольф', 'наёмник', 'глава', 'станция'), ('наёмник', 'глава', 'станция', 'технический'), ('глава', 'станция', 'технический', 'обслуживание'), ('станция', 'технический', 'обслуживание', 'бринг'), ('технический', 'обслуживание', 'бринг', 'лиманский'), ('обслуживание', 'бринг', 'лиманский', 'союз'), ('бринг', 'лиманский', 'союз', 'приграничный')]


In [50]:
with open("result_lab2/text_structures1.pkl", "wb") as f:
    pickle.dump({"BoW": word_counts1, "bigrams": bigrams1, "fourgrams": fourgrams1}, f)

In [52]:
with open("result_lab2/text_structures2.pkl", "wb") as f:
    pickle.dump({"BoW": word_counts2, "bigrams": bigrams2, "fourgrams": fourgrams2}, f)